In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [6]:
#Filter the dataset for total_votes >= 20
vote_count_df = vine_df.filter("total_votes >= 20")
vote_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [12]:
#Filter the dataset for helpful_votes >= 50%
helpful_votes_df =  vine_df.filter(vine_df['helpful_votes']/vine_df['total_votes']>=.50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
|R31673RTGEZSW7|          5|            2|          2|   N|                Y|
|R22OQLFSH42RCM|          5|            1|          1|   N|                Y|
|R12LEL4F3TSZUJ|          5|            2|          2|   N|                Y|
|R2L9XWD03072NI|          5|            1|          2|   N|                Y|
|R29SP6MBT7MXG7|          4|            3|          3|   N|                Y|
|R3OI78MWLX18YH|          1|            1|          2|   N|                Y|
| RXQWVQ6XDQ4L6|          5|            1|          2|   N|     

In [13]:
#Filter the dataset for vine = Y
vine_review_df = vine_df.filter("vine == 'Y'")
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL7RBVXGVKZSS|          4|            0|          0|   Y|                N|
|R1IB08CCW1GQ75|          5|            0|          0|   Y|                N|
|R13VNSS25T89LX|          3|            0|          1|   Y|                N|
|R34Q6NSHWU29VD|          5|            0|          1|   Y|                N|
|R3B5ZC3EF0DPBU|          4|            3|          3|   Y|                N|
| R2RN719JP825O|          5|            0|          0|   Y|                N|
|R1H9ZQWRQ7D6FC|          4|            0|          0|   Y|                N|
| RGP06KLJU55TS|          5|            1|          3|   Y|                N|
|R1TTDYWN5EANLH|          5|            0|          0|   Y|                N|
|R1NK2W3TA306O5|          4|            1|          3|   Y|     

In [14]:
#Filter the dataset for vine = N
vine_nonreview_df = vine_df.filter("vine == 'N'")
vine_nonreview_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [21]:
#Total reviews
total_reviews = vine_df.count()
total_reviews

4850360

In [15]:
#Number of 5 star reviews
five_star_reviews = vine_df.filter("star_rating == 5")
five_star_reviews.count()

3040120

In [23]:
# % of 5 Star reviews for vine reviews
five_star_reviews.filter(five_star_reviews["vine"] == "Y").count()/vine_review_df.count()


0.44454365079365077

In [24]:
# % of 5 Star reviews for vine non-reviews
five_star_reviews.filter(five_star_reviews["vine"] == "N").count()/vine_nonreview_df.count()


0.6272651438180483